In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from matplotlib import cm
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
df=pd.read_csv('datasets/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(['PassengerId','Name','Ticket'],inplace=True,axis=1)
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked'],
      dtype='object')

In [4]:
df['Embarked'].replace('S',-1,inplace=True)
df['Embarked'].replace('C',1,inplace=True)
df['Embarked'].replace('Q',0,inplace=True)
mn=df['Age'].mean()
df['Age'].fillna(mn,inplace=True)
df['Embarked'].fillna(method='ffill',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,-1.0
1,1,1,female,38.0,1,0,71.2833,C85,1.0
2,1,3,female,26.0,0,0,7.9250,NaN,-1.0
3,1,1,female,35.0,1,0,53.1000,C123,-1.0
4,0,3,male,35.0,0,0,8.0500,NaN,-1.0


In [5]:
df['Sex'].replace('female',1,inplace=True)
df['Sex'].replace('male',0,inplace=True)
df['Cabin'].fillna(0,inplace=True)
dc=df['Cabin'].unique()
dx=[dc[x] for x in range(1,len(dc))]
df['Cabin'].replace(dx,1,inplace=True)
df['Pclass'].replace([2,3],[0,-1],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,-1,0,22.0,1,0,7.2500,0,-1.0
1,1,1,1,38.0,1,0,71.2833,1,1.0
2,1,-1,1,26.0,0,0,7.9250,0,-1.0
3,1,1,1,35.0,1,0,53.1000,1,-1.0
4,0,-1,0,35.0,0,0,8.0500,0,-1.0


In [6]:
df['s+p']=df['SibSp']+df['Parch']
df.drop(['SibSp','Parch'],inplace=True,axis=1)
df.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,s+p
0,0,-1,0,22.0,7.2500,0,-1.0,1
1,1,1,1,38.0,71.2833,1,1.0,1
2,1,-1,1,26.0,7.9250,0,-1.0,0
3,1,1,1,35.0,53.1000,1,-1.0,1
4,0,-1,0,35.0,8.0500,0,-1.0,0


In [7]:
df['M_score']=3*df['Sex']+3*df['Pclass']+df['Embarked']+3*df['Cabin']+(2*df['Fare']/512)
df['m_score2']=df['Sex']+df['Cabin']
df.head()
        

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,s+p,M_score,m_score2
0,0,-1,0,22.0,7.2500,0,-1.0,1,-3.971680,0
1,1,1,1,38.0,71.2833,1,1.0,1,10.278450,2
2,1,-1,1,26.0,7.9250,0,-1.0,0,-0.969043,1
3,1,1,1,35.0,53.1000,1,-1.0,1,8.207422,2
4,0,-1,0,35.0,8.0500,0,-1.0,0,-3.968555,0


In [8]:
mn=df['M_score'].mean()
len(df.where(df['M_score']>=mn).dropna())
len(df.where((df['M_score']>=mn)&df['Survived']==1).dropna())

230

In [9]:
knn=KNC(n_neighbors=9)
x=df
y=df['Survived']
x.drop('Survived',inplace=True,axis=1)
x.columns

Index(['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Embarked', 's+p', 'M_score',
       'm_score2'],
      dtype='object')

In [10]:
knn.fit(x,y)


KNeighborsClassifier(n_neighbors=9)

In [11]:
df=pd.read_csv('datasets/test.csv')
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
df['Embarked'].replace('S',-1,inplace=True)
df['Embarked'].replace('C',1,inplace=True)
df['Embarked'].replace('Q',0,inplace=True)
mn=df['Age'].mean()
df['Age'].fillna(mn,inplace=True)
df['Embarked'].fillna(method='ffill',inplace=True)
df['Sex'].replace('female',1,inplace=True)
df['Sex'].replace('male',0,inplace=True)
df['Cabin'].fillna(0,inplace=True)
dc=df['Cabin'].unique()
dx=[dc[x] for x in range(1,len(dc))]
df['Cabin'].replace(dx,1,inplace=True)
df['Pclass'].replace([2,3],[0,-1],inplace=True)
df['s+p']=df['SibSp']+df['Parch']
df.drop(['SibSp','Parch'],inplace=True,axis=1)

df.head()


,PassengerId,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,s+p
0,892,-1,"Kelly, Mr. James",0,34.5,330911,7.8292,0,0,0
1,893,-1,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,363272,7.0000,0,-1,1
2,894,0,"Myles, Mr. Thomas Francis",0,62.0,240276,9.6875,0,0,0
3,895,-1,"Wirz, Mr. Albert",0,27.0,315154,8.6625,0,-1,0
4,896,-1,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,3101298,12.2875,0,-1,2


In [13]:
x_predict=pd.DataFrame(df['PassengerId'])
x_test=df.drop(['PassengerId','Name','Ticket'],axis=1)
x_test['Fare'].replace(np.nan,x_test['Fare'].mean(),inplace=True)
x_test['M_score']=3*x_test['Sex']+3*x_test['Pclass']+x_test['Embarked']+3*x_test['Cabin']+(2*x_test['Fare']/512)
x_test['m_score2']=x_test['Sex']+x_test['Cabin']
x_test.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,s+p,M_score,m_score2
0,-1,0,34.5,7.8292,0,0,0,-2.969417,0
1,-1,1,47.0,7.0000,0,-1,1,-0.972656,1
2,0,0,62.0,9.6875,0,0,0,0.037842,0
3,-1,0,27.0,8.6625,0,-1,0,-3.966162,0
4,-1,1,22.0,12.2875,0,-1,2,-0.952002,1


In [14]:
k=[]
for x in range(len(x_test)):
    predict=knn.predict([x_test.loc[x]])
    k.append(predict[0])
pr=np.array(k)
pr
    
    
    

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [15]:
x_predict['Survived']=pr
x_predict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [16]:
x_predict.to_csv('datasets/predict.csv',index=False)

In [17]:
krange=range(1,9)
scrs=[]
for k in krange:
    knn=KNC(n_neighbors=k)
    knn.fit(x_train,y_train)
    scrs.append(knn.score(x_test,y_test))
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(krange,scrs)
plt.xticks([0,1,2,3,4,5,6,7,8,9]);

NameError: name 'x_train' is not defined

In [ ]:
x_test.isnull().sum()